# Kotlin Coroutines: launch vs async

A quick reference table highlighting the practical differences between the two coroutine builders.


In [4]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")


## Imports

We import the core coroutines package so that launch, async, Job, Deferred, etc. are available.


In [5]:
import kotlinx.coroutines.*


## launch vs async — cheat sheet

| Aspect | launch | async |
|---|---|---|
| Return type | Job | Deferred<T> (also a Job) |
| Primary intent | Fire-and-forget work tied to a scope | Concurrently produce a value (future/promise) |
| How to wait | job.join() (does not throw the exception) | deferred.await() (returns T or throws) |
| How to get result | No direct result | await() to get T |
| Exception propagation | Unhandled exception is immediately propagated to the parent and handled by CoroutineExceptionHandler | Exception is captured and rethrown on await(); if never awaited, it still cancels the parent (unless using supervisor) but won’t trigger CoroutineExceptionHandler by itself |
| Try/catch location | Wrap the body of launch or use a CoroutineExceptionHandler | Surround await() with try/catch, or use supervisor constructs |
| Structured concurrency | Child is tied to its parent scope; parent waits for job completion on cancellation | Same; Deferred is also a child tied to its parent scope |
| Cancellation | job.cancel() cancels the work; parent/child cancellation propagates | deferred.cancel() cancels; same propagation rules |
| Start options | CoroutineStart.DEFAULT (eager) by default; LAZY supported via start=CoroutineStart.LAZY | Same start options as launch |
| Combining/parallelism | Use multiple launch jobs and join them; no values to combine | Ideal for parallel computations; use await() / awaitAll() to combine results |
| Typical use cases | UI updates, logging, fire-and-forget tasks, starting child coroutines in a scope | Fetching/combining data, parallel computations returning values |
| Handling completion | job.invokeOnCompletion { … } | deferred.invokeOnCompletion { … } or await() |
| Treat as Job | Yes (it is a Job) | Yes (Deferred<T> extends Job) |


### Quick examples

- launch (fire-and-forget):


In [6]:
runBlocking {
    val job = launch {
        println("Doing some work…")
        delay(100)
        println("Done")
    }
    job.join() // wait until it completes; doesn’t throw on failure
}

Doing some work…
Done


- async (producing a value):

In [7]:
suspend fun compute(i: Int): Int { delay(100); return i * 2 }

runBlocking {
    val a = async { compute(21) }
    val b = async { compute(21) }
    val sum = a.await() + b.await()
    println(sum) // 84
}

84
